In [ ]:
import nltk
import collections
import re, string
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

#from PIL import Image
#from datetime import datetime
from string import punctuation
from nltk.corpus import stopwords
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

In [ ]:
#arch = input("Arquivo: ")
arch = "EXTRACAO.csv" #indicar o arquivo .csv com a extração

table = pd.read_csv(arch)
table = table.sort_values(by=['date','conversation_id','turn_counter'], ascending=True).reset_index(drop=True)

table.sort_values('conversation_id')

In [ ]:
#table.plot.scatter(x='confidence', y='turn_counter', figsize=(20, 10), ylim = [0, 2])

# Mensagens por dia

In [ ]:
daily = table.groupby('date').agg({'conversation_id': 'count'}).rename(columns={"conversation_id": "mensagens"})
daily_conversations = table.groupby('date').agg({'conversation_id': 'nunique'}).rename(columns={"conversation_id": "conversas"})
daily = daily.join(daily_conversations)
daily["média"] = round(daily['mensagens']/daily['conversas'], 1)
daily


# Transferências por dia

In [ ]:
human = table.groupby(['date', 'bot_message'], as_index=False).agg({'conversation_id': 'count'})
human2 = human[human.bot_message == '[[REDIRECT_CUSTHELP]]'].drop('bot_message', axis=1).reset_index(drop=True).set_index('date')
human2

# <span style="color:blue">Conversas</span> X <span style="color:red">Transferências</span>

In [ ]:
daily_conversations['conversas'].plot.bar(figsize=(25,5), color='royalblue')
human2['conversation_id'].plot.bar(color='indianred')

#title = str(plt.title('Conversas X Transferências'))
#plt.xlabel('Data')
#plt.ylabel('Conversas + Transferências')
#plt.savefig(title + '.png')

In [ ]:
joined = daily_conversations.join(human2['conversation_id'])

joined['% Transferidos'] = round(joined['conversation_id']/joined['conversas'], 4) * 100
joined['% Retidos'] = 100  - joined['% Transferidos']
joined.rename(columns={"conversation_id": "transferências"})[:10]

In [ ]:
joined['% Retidos'].plot.line(color='olive', figsize=(20,5), xticks=range(1, len(joined.index)), rot=270, grid=True, marker="o")

title = str(plt.title('Taxa de Retenção'))
plt.xlabel('Data')
plt.ylabel('Retidos')
#plt.savefig(title + '.png')

In [ ]:
joined['conversation_id'].sum()/joined['conversas'].sum() * 100

# Acionamento de intenções (geral)

In [ ]:
intents = table.groupby('intent').agg({'conversation_id': 'count', 'confidence': 'mean'})
intents['%'] = round(intents['conversation_id']/intents['conversation_id'].sum(), 3) * 100
intents.sort_values('conversation_id', ascending=False)

In [ ]:
#intents.plot.scatter(y='%', x='confidence', color='teal')

# Acionamento de intenções (1ª interação)

In [ ]:
first_turn = table.groupby(['turn_counter', 'intent'], as_index=False).agg({'conversation_id': 'nunique', 'confidence':'mean'})
first_turn2 = first_turn[first_turn.turn_counter == 1].drop('turn_counter', axis=1).reset_index(drop=True).set_index('intent')
first_turn2['%'] = round(first_turn2['conversation_id']/first_turn2['conversation_id'].sum(), 3) * 100
first_turn2.sort_values('conversation_id', ascending=False)

# Indicadores de tendências por turno 

In [ ]:
def flow_matrix_x(intent):
    flow_cid = table[table.turn_counter == 1][table.intent==intent]
    cid_set = set(flow_cid['conversation_id'])
    cid = table['conversation_id']
    cid_count = collections.Counter(cid)
    cid_dic = dict()

    for x in cid_count:
        if x in cid_set:
            cid_dic[x]=cid_count[x]
        
    dic_count = collections.Counter([cid_dic[key] for key in cid_dic])
    list_count = list(dic_count)
    list_count.sort()
    return list_count

def flow_matrix_y(intent):
    flow_cid = table[table.turn_counter == 1][table.intent==intent]
    cid_set = set(flow_cid['conversation_id'])
    cid = table['conversation_id']
    cid_count = collections.Counter(cid)
    cid_dic = dict()

    for x in cid_count:
        if x in cid_set:
            cid_dic[x]=cid_count[x]
        
    dic_count = collections.Counter([cid_dic[key] for key in cid_dic])
    list_count = list(dic_count)
    list_count.sort()

    graphic = [dic_count[key] for key in list_count]
    graphic
    return graphic

In [ ]:
plt.figure(figsize=(20, 10))

plt.plot(flow_matrix_x('#consultar'), flow_matrix_y('#consultar'), 
         label='Consultar', marker='o', linewidth=2, markersize=12)
plt.plot(flow_matrix_x('#recomendar'), flow_matrix_y('#recomendar'), 
         label='Recomendar', marker='o', linewidth=2, markersize=12)
plt.plot(flow_matrix_x('#especificar'), flow_matrix_y('#especificar'), 
         label='Especificar', marker='o', linewidth=2, markersize=12)
plt.plot(flow_matrix_x('#barganhar'), flow_matrix_y('#barganhar'), 
         label='Barganhar', marker='o', linewidth=2, markersize=12)

plt.axis([0, 12, 0, 2100])
plt.grid()

plt.legend()
title = str(plt.title('Análise métrica de abandono por turno'))
plt.xlabel('Turno')
plt.ylabel('Conversas')
plt.show()
#plt.savefig(title + '.png')

# Mensagens e transferência por turno

In [ ]:
turns = table.groupby(['turn_counter']).agg({'conversation_id': 'nunique'}).rename(columns={"conversation_id": "conversas"})
retention = table.groupby(['bot_message', 'turn_counter'], as_index=False).agg({'conversation_id': 'nunique'})
retention = retention[retention.bot_message == '[[REDIRECT_CUSTHELP]]'].drop('bot_message', axis=1).reset_index(drop=True).set_index('turn_counter').rename(columns={"conversation_id": "transferências"})
media_turnos = turns.join(retention)
media_turn_trasnf = media_turnos["transferências"].sum() / media_turnos["conversas"].sum() *100

print('Média de transferências por turno:', round(media_turn_trasnf))

### Mensagens x Turno

In [ ]:
ab = turns.plot.line(color='lime', xticks=turns.index, xlim = [1, 10], grid=True, figsize=(10,5))
retention.plot.line(color='firebrick', xticks=retention.index, xlim = [1, 10], grid=True, figsize=(10,5), ax=ab)

title = str(plt.title('Mensagens x Turno'))
plt.xlabel('Turno')
plt.ylabel('Mensagens + Transferências')
#plt.savefig(title + '.png')

### Transferências x Turno

In [ ]:
retention.plot.line(color='firebrick', xticks=retention.index, xlim = [1, 10], grid=True, figsize=(10, 5))

title = str(plt.title('Transferências x Turno'))
plt.xlabel('Turno')
plt.ylabel('Transferências')
#plt.savefig(title + '.png')

# Irrelevant

In [ ]:
irrelevante_table = pd.DataFrame(table[table.confidence <= 0.2]['input']).join(table[table.confidence <= 0.2]['intent']).join(table[table.confidence <= 0.2]['confidence']).reset_index(drop=True)
irrelevante_table[:1000]

In [ ]:
agg_it = irrelevante_table.groupby('intent').agg({'confidence':'count'}).sort_values('confidence', ascending=False)
agg_it['%'] = round(agg_it['confidence']/agg_it['confidence'].sum(), 3) * 100
agg_it

# Nuvem de Palavras

In [ ]:
msg = list(table['input'])
msg_list = []
for i in msg:
    for j in i.split():
        msg_list.append(j.lower())

In [ ]:
word_cont = collections.Counter(msg_list) #contagem de frequencias
limit_ranking = word_cont.most_common(1000000) #definindo o limite a ser mostrado no ranking (aqui a necessidade de limitar se deve a identação para criação do df)
df = pd.DataFrame(limit_ranking, columns=['Palavras', 'Qtda']) 

In [ ]:
# lista de stopword
stoplist = ['da', 'meu', 'em', 'você', 'de', 'ao', 'os', 'deu', 'sim', 'não', 'gmail', 'hotmail', ' ', '', 
            'voltar', 'menu', 'retornar', 'alarme','capacidade','cesto','quadrichama','duplachama', 
            'ciclo_edredom', 'brancomaisbranco','programa', 'coresmaisvivas', 'turbo_control', 'extrafrio', 
            'espaco_adapt', 'acesso_rapido', 'compartimento', 'twist_ice', 'latas_e_longnecks', 'conservacao', 
            'telefone', 'numero', 'contato', 'televenda', 'controle', 'degelo', 'frostfree', 'design', 
            'multidispenser', 'dual_dispenser', 'gratuidade', 'promocao', 'cupom', 'economia', 'desconto', 
            'valor', 'etapa', 'expirado', 'boca','forno_duplo', 'filtro_antifiapos', 'smartbar', 
            'funcao_turbo', 'turbo_ice', 'icemaker', 'enxague_duplo', 'enxague_antialergico', 'funcao_enxague', 
            'gavetao', 'grill', 'fogao', 'lavadora', 'geladeira', 'iluminacao_led', 'inverse', 'vidro', 
            'mesa', 'painel', 'prateleira', '6_bocas_antes', '6_bocas_depois', '4_bocas_depois', 
            '5_bocas_depois', '5_bocas_antes', '4_bocas_antes', 'sem_forno_duplo_depois', 
            'sem_forno_duplo_antes', 'com_forno_duplo_depois', 'com_forno_duplo_antes', 'branca', 'preta', 
            'inox', 'frostfree_sim_depois', 'frostfree_nao_antes','frostfree_nao_depois', 
            'frostfree_sim_antes', 'cinco_mais', 'um_dois', 'tres_quatro', 'fr_no_alto', 'fr_ao_lado', 
            'fr_embaixo', 'com_restricao', 'sem_restricao', 'pouca_qtd', 'media_qtd', 'muita_qtd', '3_vez', 
            '2_vez', '1_vez', 'sku', 'temperatura', 'timer', '110v_lavadora', '220v_geladeira', '110v_geladeira', 
            '110v_fogao', '110v', '220v_lavadora', '220v_fogao', '220v', 'continuar', 'nao', 'frost_free', 
            'prosseguir_frost_free', 'pra', 'para', 'ja', 'oi', 'ola', ]

stopwords = set(stopwords.words('portuguese') + list(punctuation) +  list(STOPWORDS) + stoplist)

In [ ]:
for index, row in df.iterrows():
    if row['Palavras'] in stopwords:
        df = df.drop(index)
        
df[:10]

In [ ]:
words_dic = {}
words_dic_sum = df["Qtda"].sum()
for index, row in df.iterrows():
    words_dic[re.sub('[%s]' % re.escape(string.punctuation), '', row['Palavras'])] = float(row['Qtda'] / words_dic_sum)

In [ ]:
#gerar uma wordclou
wordcloud = WordCloud(stopwords=stopwords,
                      max_words=1000,
                      max_font_size=70, 
                      color_func=lambda *args, **kwargs: 'gold', 
                      background_color='indigo').generate_from_frequencies(words_dic)

In [ ]:
# mostrar a imagem final
plt.figure(figsize=[20, 20])
plt.imshow(wordcloud)
plt.axis("off")
plt.show()
#plt.savefig('Nuvem de Palavras.png')